![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 2: RAG Fundamentals and Implementation

## Workshop: Context Engineering Matters

---

## Learning Objectives

By the end of this module, you will:

1. **Understand** what RAG is and why it matters for context engineering
2. **Implement** vector embeddings and semantic search with Redis
3. **Build** a complete RAG pipeline using HierarchicalCourseManager
4. **Apply** progressive disclosure for efficient context retrieval
5. **Compare** poor vs. well-engineered context approaches

---

## Prerequisites

- Completed Module 1: Introduction to Context Engineering
- Redis Stack running (with vector search capability)
- OpenAI API key configured
- Course data loaded in Redis

---

## Part 1: What is RAG?

**Retrieval-Augmented Generation (RAG)** is a technique that enhances LLM responses by:

1. **Retrieving** relevant information from external sources
2. **Augmenting** the prompt with this retrieved context
3. **Generating** responses grounded in factual data

### Why RAG Matters

| Problem | RAG Solution |
|---------|-------------|
| LLMs have knowledge cutoff dates | Retrieve current information |
| LLMs can hallucinate facts | Ground responses in real data |
| LLMs don't know your domain | Inject domain-specific knowledge |
| Context windows are limited | Retrieve only relevant information |

### The RAG Pipeline

```
User Query → Embed Query → Search Vector DB → Retrieve Documents → Assemble Context → Generate Response
```

Let's build each component step by step.

---

## Part 2: Setup and Configuration

In [1]:
# Standard library imports
import os
import sys
from pathlib import Path

# Add project root to path for imports
# Handle both running from workshop/ directory and from project root
if Path.cwd().name == "workshop":
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables from project root
from dotenv import load_dotenv
env_path = project_root / ".env"
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
required_vars = ["OPENAI_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"""
⚠️  Missing required environment variables: {', '.join(missing_vars)}

Please create a .env file in the project root with:
OPENAI_API_KEY=your_openai_api_key

For Redis setup:
- Local: docker run -d -p 6379:6379 redis/redis-stack-server:latest
- Cloud: https://redis.com/try-free/
""")
    raise SystemExit(1)

print("✅ Environment variables loaded")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")

# Third-party imports
import numpy as np
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage

# Project imports
from redis_context_course.redis_config import redis_config
from redis_context_course.hierarchical_manager import HierarchicalCourseManager
from redis_context_course.hierarchical_context import HierarchicalContextAssembler

print("✅ Imports successful!")

✅ Environment variables loaded
   OPENAI_API_KEY: ✓ Set


✅ Imports successful!


In [2]:
# Initialize clients
openai_client = OpenAI()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Initialize Redis connection (redis_config.redis_client is a property)
redis_client = redis_config.redis_client

# Initialize HierarchicalCourseManager for two-tier retrieval
hierarchical_manager = HierarchicalCourseManager(redis_client=redis_client)
context_assembler = HierarchicalContextAssembler()

# Verify connection
try:
    redis_client.ping()
    print("""✅ Redis connection successful!
✅ HierarchicalCourseManager initialized
✅ HierarchicalContextAssembler initialized""")
except Exception as e:
    print(f"❌ Redis connection failed: {e}")

✅ Redis connection successful!
✅ HierarchicalCourseManager initialized
✅ HierarchicalContextAssembler initialized


---

## Part 3: Understanding Vector Embeddings

### What are Embeddings?

Embeddings are **numerical representations** of text that capture semantic meaning:

- Similar concepts have similar embeddings
- We can measure similarity using cosine similarity
- This enables **semantic search** (finding by meaning, not keywords)

### Embedding Dimensions

| Model | Dimensions | Use Case |
|-------|------------|----------|
| text-embedding-3-small | 1536 | General purpose, cost-effective |
| text-embedding-3-large | 3072 | Higher accuracy, more expensive |
| text-embedding-ada-002 | 1536 | Legacy model |

Let's see embeddings in action:

In [3]:
# Create embeddings for sample texts
sample_texts = [
    "Introduction to machine learning and neural networks",
    "Deep learning fundamentals with Python",
    "Advanced calculus and differential equations",
    "Redis database administration and optimization",
]

# Generate embeddings
response = openai_client.embeddings.create(
    model="text-embedding-3-small",
    input=sample_texts
)

# Extract embedding vectors
embeddings_list = [item.embedding for item in response.data]

print(f"""Vector Embeddings Demo
{'=' * 60}
Number of texts: {len(sample_texts)}
Embedding dimensions: {len(embeddings_list[0])}

Sample embedding (first 10 values):
{embeddings_list[0][:10]}...""")

19:13:17 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Vector Embeddings Demo
Number of texts: 4
Embedding dimensions: 1536

Sample embedding (first 10 values):
[-0.030342772603034973, -0.013117661699652672, 0.0014119355473667383, -0.03249717131257057, 0.03269851580262184, -0.003513479605317116, -0.0007751803495921195, 0.04941019415855408, -0.02436280995607376, 0.07264547049999237]...


### Cosine Similarity

Cosine similarity measures the angle between two vectors:
- **1.0** = Identical meaning
- **0.0** = Unrelated
- **-1.0** = Opposite meaning (rare in practice)

Let's compare our sample texts:

In [4]:
def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Compare all pairs
print(f"""Semantic Similarity Matrix
{'=' * 60}""")

for i, text1 in enumerate(sample_texts):
    print(f"\n[{i}] {text1[:50]}...")
    for j, text2 in enumerate(sample_texts):
        if i < j:
            similarity = cosine_similarity(embeddings_list[i], embeddings_list[j])
            print(f"    vs [{j}]: {similarity:.4f}")

Semantic Similarity Matrix

[0] Introduction to machine learning and neural networ...
    vs [1]: 0.5249
    vs [2]: 0.3053
    vs [3]: 0.0955

[1] Deep learning fundamentals with Python...
    vs [2]: 0.2684
    vs [3]: 0.1511

[2] Advanced calculus and differential equations...
    vs [3]: 0.1426

[3] Redis database administration and optimization...


### Key Insight: Semantic vs. Keyword Search

Notice how:
- "machine learning" and "deep learning" have HIGH similarity (related topics)
- "machine learning" and "calculus" have LOWER similarity (different domains)
- "Redis database" is distinct from all ML topics

This is the power of semantic search - it understands **meaning**, not just keywords.

---

## Part 4: Progressive Disclosure with HierarchicalCourseManager

### The Two-Tier Architecture

The `HierarchicalCourseManager` implements **progressive disclosure**:

```
Tier 1: Course Summaries (lightweight, fast search)
    ↓
Tier 2: Course Details (comprehensive, on-demand)
```

**Why this matters:**
- Search across ALL courses using lightweight summaries
- Fetch full details ONLY for the most relevant matches
- Efficient token usage (don't load everything upfront)

### Key Methods

| Method | Purpose | Returns |
|--------|---------|--------|
| `search_summaries()` | Tier 1: Fast vector search | List of CourseSummary |
| `fetch_details()` | Tier 2: Get full course info | List of CourseDetails |
| `hierarchical_search()` | Combined two-stage retrieval | (summaries, details) |

In [5]:
# Demonstrate Tier 1: Search Summaries
query = "machine learning courses for beginners"

print(f"""Tier 1: Searching Course Summaries
{'=' * 60}
Query: "{query}"
""")

# Search summaries (lightweight)
summaries = await hierarchical_manager.search_summaries(
    query=query,
    limit=5
)

print(f"Found {len(summaries)} matching courses:\n")
for i, summary in enumerate(summaries, 1):
    print(f"""{i}. {summary.course_code}: {summary.title}
   Department: {summary.department} | Level: {summary.difficulty_level.value}
   {summary.short_description[:100]}...
""")

Tier 1: Searching Course Summaries
Query: "machine learning courses for beginners"



19:13:17 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:17 redis_context_course.hierarchical_manager INFO   Created summary index: course_summaries


19:13:17 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: machine learning courses for beginners


Found 0 matching courses:



In [6]:
# Demonstrate Tier 2: Fetch Details for Top Matches
top_course_codes = [s.course_code for s in summaries[:2]]

print(f"""Tier 2: Fetching Full Details
{'=' * 60}
Fetching details for: {top_course_codes}
""")

details = await hierarchical_manager.fetch_details(top_course_codes)

for detail in details:
    print(f"""
{'─' * 60}
{detail.course_code}: {detail.title}
Instructor: {detail.instructor}

Description:
{detail.full_description[:300]}...

Learning Objectives:""")
    for obj in detail.learning_objectives[:3]:
        print(f"  • {obj}")
    print(f"""
Syllabus: {detail.syllabus.total_weeks} weeks
Assignments: {detail.get_total_assignments()} total ({detail.get_total_points()} points)""")

Tier 2: Fetching Full Details
Fetching details for: []

19:13:18 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


In [7]:
# Demonstrate Combined Hierarchical Search
query = "advanced Redis data structures"

print(f"""Hierarchical Search (Two-Stage Retrieval)
{'=' * 60}
Query: "{query}"
""")

summaries, details = await hierarchical_manager.hierarchical_search(
    query=query,
    summary_limit=5,  # Get 5 summaries
    detail_limit=2    # But only 2 full details
)

print(f"""Stage 1: Retrieved {len(summaries)} summaries
Stage 2: Fetched {len(details)} detailed courses

All Matching Courses (Summaries):""")
for i, s in enumerate(summaries, 1):
    marker = "★" if i <= len(details) else "○"
    print(f"  {marker} {s.course_code}: {s.title}")

print(f"\n★ = Full details available | ○ = Summary only")

Hierarchical Search (Two-Stage Retrieval)
Query: "advanced Redis data structures"

19:13:18 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'advanced Redis data structures' (summaries=5, details=2)


19:13:18 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:18 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: advanced Redis data structures


19:13:18 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:18 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


Stage 1: Retrieved 0 summaries
Stage 2: Fetched 0 detailed courses

All Matching Courses (Summaries):

★ = Full details available | ○ = Summary only


---

## Part 5: Building the Complete RAG Pipeline

Now let's combine everything into a complete RAG pipeline: Retrieval → Context Assembly → Generation.

### The RAG Flow

```
User Query
    ↓
1. Hierarchical Search (retrieve summaries → fetch details)
    ↓
2. Context Assembly (combine system + user + retrieved context)
    ↓
3. LLM Generation (create personalized response)
```

Let's implement each step:

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

print("✅ LLM initialized (gpt-4o-mini)")

✅ LLM initialized (gpt-4o-mini)


### Step 5.1: Retrieval Function with Hierarchical Search

First, let's create a function to retrieve relevant courses using hierarchical search:

In [9]:
async def retrieve_courses_hierarchical(
    query: str,
    summary_limit: int = 5,
    detail_limit: int = 3
):
    """
    Retrieve relevant courses using hierarchical search.

    Args:
        query: User's search query
        summary_limit: Number of summaries to retrieve
        detail_limit: Number of full details to fetch

    Returns:
        Tuple of (summaries, details)
    """
    summaries, details = await hierarchical_manager.hierarchical_search(
        query=query,
        summary_limit=summary_limit,
        detail_limit=detail_limit
    )
    return summaries, details


# Test retrieval
test_query = "I want to learn about data structures"
test_summaries, test_details = await retrieve_courses_hierarchical(test_query)

print(f"🔍 Retrieved for: '{test_query}'")
print(f"   Summaries: {len(test_summaries)}")
print(f"   Details: {len(test_details)}")
for s in test_summaries:
    print(f"   - {s.course_code}: {s.title}")

19:13:18 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I want to learn about data structures' (summaries=5, details=3)


19:13:19 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:19 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: I want to learn about data structures


19:13:19 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:19 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


🔍 Retrieved for: 'I want to learn about data structures'
   Summaries: 0
   Details: 0


### Step 5.2: Context Assembly Function

Now let's assemble context from multiple sources (system + user + retrieved):

In [10]:
def assemble_rag_context(
    user_query: str,
    summaries: list,
    details: list,
    user_profile: dict = None
):
    """
    Assemble context from multiple sources for the LLM.

    This implements the context engineering principles from Module 1:
    - System Context: AI role and instructions
    - User Context: Student profile and preferences
    - Retrieved Context: Relevant courses from hierarchical search
    """

    # System Context: Define the AI's role
    system_context = """You are a Redis University course advisor.

Your role:
- Help students find courses that match their interests and goals
- Provide personalized recommendations based on student profiles
- Explain course prerequisites and learning paths
- Be encouraging and supportive

Guidelines:
- Only recommend courses from the provided course list
- Consider student's difficulty level preferences
- Explain your reasoning for recommendations
- Be concise but informative
"""

    # User Context: Student profile (if provided)
    user_context = ""
    if user_profile:
        user_context = f"""
Student Profile:
- Name: {user_profile.get('name', 'Student')}
- Major: {user_profile.get('major', 'Undeclared')}
- Year: {user_profile.get('year', 'N/A')}
- Interests: {', '.join(user_profile.get('interests', []))}
- Preferred Difficulty: {user_profile.get('preferred_difficulty', 'any')}
- Preferred Format: {user_profile.get('preferred_format', 'any')}
"""

    # Retrieved Context: Use HierarchicalContextAssembler
    retrieved_context = context_assembler.assemble_hierarchical_context(
        summaries=summaries,
        details=details,
        query=user_query
    )

    # Combine all context
    full_context = system_context
    if user_context:
        full_context += user_context
    full_context += "\n" + retrieved_context

    return full_context


# Test context assembly
test_profile = {
    "name": "Sarah Chen",
    "major": "Computer Science",
    "year": "Junior",
    "interests": ["machine learning", "data science"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "online",
}

assembled_context = assemble_rag_context(
    user_query=test_query,
    summaries=test_summaries,
    details=test_details,
    user_profile=test_profile,
)

print("✅ Context assembled")
print(f"   Total length: {len(assembled_context)} characters")
print(f"   Includes: System + User + Retrieved context (hierarchical)")

✅ Context assembled
   Total length: 769 characters
   Includes: System + User + Retrieved context (hierarchical)


In [11]:
print(f"Observe the assembled context: \n\n{assembled_context}")

Observe the assembled context: 

You are a Redis University course advisor.

Your role:
- Help students find courses that match their interests and goals
- Provide personalized recommendations based on student profiles
- Explain course prerequisites and learning paths
- Be encouraging and supportive

Guidelines:
- Only recommend courses from the provided course list
- Consider student's difficulty level preferences
- Explain your reasoning for recommendations
- Be concise but informative

Student Profile:
- Name: Sarah Chen
- Major: Computer Science
- Year: Junior
- Interests: machine learning, data science
- Preferred Difficulty: intermediate
- Preferred Format: online

# Course Search Results for: I want to learn about data structures

## Overview of All Matches

Found 0 relevant courses:



**🎁 Bonus:** Can you identify the different parts of the context from what we learned in Module 1?

**✅ Answer:** Yes! Looking at the assembled context above, we can identify all three context types from Module 1:

1. **System Context** (Static)
   - The first section: "You are a Redis University course advisor..."
   - Defines the AI's role, responsibilities, and guidelines
   - Remains the same for all queries
   - Sets behavioral instructions and constraints

2. **User Context** (Dynamic, User-Specific)
   - The "Student Profile" section
   - Contains Sarah Chen's personal information: major, year, interests, preferences
   - Changes based on who is asking the question
   - Enables personalized recommendations

3. **Retrieved Context** (Dynamic, Query-Specific)
   - The "Relevant Courses" section with summaries and details
   - Uses progressive disclosure (summaries for all, details for top matches)
   - Changes based on the specific query
   - Provides the factual information the LLM needs to answer

Notice how all three work together: System Context tells the AI **how to behave**, User Context tells it **who it's helping**, and Retrieved Context provides **what information is relevant**. This is RAG in action!

### Step 5.3: Generation Function

Finally, let's generate a response using the assembled context:

In [12]:
async def generate_response(user_query: str, context: str):
    """
    Generate LLM response using assembled context.

    Args:
        user_query: User's question
        context: Assembled context (system + user + retrieved)

    Returns:
        LLM response string
    """
    messages = [SystemMessage(content=context), HumanMessage(content=user_query)]

    response = await llm.ainvoke(messages)
    return response.content


# Test generation
response = await generate_response(test_query, assembled_context)

print("\n🤖 Generated Response:\n")
print(response)

19:13:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Generated Response:

It looks like there are currently no courses available that focus specifically on data structures. However, since you are interested in machine learning and data science, I recommend exploring courses that cover those topics. 

Would you like me to assist you in finding courses related to machine learning or data science that align with your intermediate difficulty preference? These subjects often involve a strong understanding of data structures, which could be beneficial for your studies in computer science. Let me know how you would like to proceed!


### 🎯 Understanding the Generated Response

Notice how the LLM's response demonstrates effective context engineering:

**👤 Personalization from User Context:**
- Addresses Sarah by name
- References her intermediate difficulty preference
- Acknowledges her online format preference
- Connects to her interests (machine learning and data science)

**📚 Accuracy from Retrieved Context:**
- Recommends courses from the retrieved list
- Provides correct course details (difficulty, format, credits, description)
- Mentions prerequisites accurately

**🤖 Guidance from System Context:**
- Acts as a supportive advisor
- Explains reasoning for the recommendation
- Stays within the provided course list

This is the power of RAG: the LLM generates a response that is **personalized** (User Context), **accurate** (Retrieved Context), and **helpful** (System Context). Without RAG, the LLM would either hallucinate course details or provide generic advice.

---

## Part 6: Complete RAG Function

Let's combine all three steps into a single, reusable RAG function:

In [13]:
async def hierarchical_rag_query(
    user_query: str,
    user_profile: dict = None,
    summary_limit: int = 5,
    detail_limit: int = 3
):
    """
    Complete RAG pipeline with hierarchical retrieval: Retrieve → Assemble → Generate

    Args:
        user_query: User's question
        user_profile: Optional student profile
        summary_limit: Number of summaries to retrieve
        detail_limit: Number of full details to fetch

    Returns:
        Tuple of (response, summaries, details)
    """
    # Step 1: Retrieve relevant courses (hierarchical)
    summaries, details = await retrieve_courses_hierarchical(
        user_query, summary_limit, detail_limit
    )

    # Step 2: Assemble context
    context = assemble_rag_context(user_query, summaries, details, user_profile)

    # Step 3: Generate response
    response = await generate_response(user_query, context)

    return response, summaries, details


# Test the complete RAG pipeline
print("=" * 60)
print("COMPLETE HIERARCHICAL RAG PIPELINE TEST")
print("=" * 60)
print()

query = "I'm interested in learning about databases and data management"
profile = {
    "name": "Alex Johnson",
    "major": "Data Science",
    "year": "Sophomore",
    "interests": ["databases", "data analysis", "SQL"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "hybrid",
}

print(f"Query: {query}")
print()
print(f"Student: {profile['name']} ({profile['major']}, {profile['year']})")
print()

response, summaries, details = await hierarchical_rag_query(query, profile)

print("Retrieved Courses:")
for i, s in enumerate(summaries, 1):
    marker = "★" if i <= len(details) else "○"
    print(f"   {marker} {s.course_code}: {s.title}")
print()

print("AI Response:")
print(response)

COMPLETE HIERARCHICAL RAG PIPELINE TEST

Query: I'm interested in learning about databases and data management

Student: Alex Johnson (Data Science, Sophomore)

19:13:22 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm interested in learning about databases and data management' (summaries=5, details=3)


19:13:23 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:23 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: I'm interested in learning about databases and data management


19:13:23 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:23 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:13:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Retrieved Courses:

AI Response:
It seems that there are currently no specific courses available that match your interests in databases and data management. However, I encourage you to explore other related topics or areas that might align with your goals in data science.

Given your major and your interest in SQL and data analysis, I recommend looking for courses that cover:

1. **Data Warehousing**: This will deepen your understanding of how databases are structured for efficient data analysis.
2. **SQL for Data Science**: Courses that focus on SQL will enhance your skills in querying databases and managing data effectively.

If you have access to other platforms or institutions, consider checking their offerings as well. Don't hesitate to reach out if you need further assistance or have other interests you'd like to explore!


### 🎯 Why This Complete RAG Function Matters

The `hierarchical_rag_query()` function encapsulates the entire RAG pipeline in a single, reusable interface. This is important because:

**1. Simplicity:** One function call handles retrieval → assembly → generation
- No need to manually orchestrate the three steps
- Clean API for building applications

**2. Consistency:** Every query follows the same pattern
- Ensures all three context types are always included
- Reduces errors from missing context

**3. Efficiency:** Progressive disclosure saves tokens
- Summaries for all matches (lightweight)
- Full details only for top matches (on-demand)
- Optimal token usage

**4. Production-Ready:** This pattern scales to real applications
- In Module 4, we'll add memory (conversation history)
- In Module 5, we'll add tools (course enrollment, prerequisites checking)
- The core RAG pattern remains the same

This is the foundation you'll build on throughout the rest of the workshop.

---

## Part 7: Try Different Queries

Let's test our RAG system with various queries to see how it handles different scenarios:

In [14]:
# Test 1: Beginner looking for programming courses
print("=" * 60)
print("TEST 1: Beginner Programming")
print("=" * 60)
print()

query1 = "I'm new to programming and want to start learning"
profile1 = {
    "name": "Maria Garcia",
    "major": "Undeclared",
    "year": "Freshman",
    "interests": ["programming", "technology"],
    "preferred_difficulty": "beginner",
    "preferred_format": "online",
}

response1, summaries1, details1 = await hierarchical_rag_query(query1, profile1)
print(f"\nQuery: {query1}\n")
print("\nAI Response:\n")
print(response1)

TEST 1: Beginner Programming

19:13:27 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm new to programming and want to start learning' (summaries=5, details=3)


19:13:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:27 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: I'm new to programming and want to start learning


19:13:27 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:27 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:13:31 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: I'm new to programming and want to start learning


AI Response:

It seems that there are currently no specific courses available that match your interests in programming and technology at a beginner level. However, I encourage you to explore other resources and platforms that may offer foundational programming courses. 

Since you are a freshman and undeclared in your major, this could also be a great opportunity to explore introductory courses in computer science or programming languages like Python, JavaScript, or HTML/CSS, if you come across them in the future.

If you have any other interests or specific areas you'd like to explore further, please let me know, and I can assist you in finding resources that align with your goals! Keep up the enthusiasm for learning, and I'm here to support you!


In [15]:
# Test 2: Advanced student looking for specialized courses
print("=" * 60)
print("TEST 2: Advanced Machine Learning")
print("=" * 60)
print()

query2 = "I want advanced courses in machine learning and AI"
profile2 = {
    "name": "David Kim",
    "major": "Computer Science",
    "year": "Senior",
    "interests": ["machine learning", "AI", "research"],
    "preferred_difficulty": "advanced",
    "preferred_format": "in-person",
}

response2, summaries2, details2 = await hierarchical_rag_query(query2, profile2)
print(f"\nQuery: {query2}\n")
print("\nAI Response:\n")
print(response2)

TEST 2: Advanced Machine Learning

19:13:31 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I want advanced courses in machine learning and AI' (summaries=5, details=3)


19:13:31 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:31 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: I want advanced courses in machine learning and AI


19:13:31 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:31 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:13:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: I want advanced courses in machine learning and AI


AI Response:

It seems that there are currently no advanced courses in machine learning and AI available in the provided course list. However, I encourage you to explore other related topics or perhaps consider foundational courses to enhance your understanding, which could benefit your research in the future.

If there are any specific subjects or other areas of interest you would like me to look into, please let me know! I'm here to help you find the right path that aligns with your goals.


In [16]:
# Test 3: Business student looking for relevant courses
print("=" * 60)
print("TEST 3: Business Analytics")
print("=" * 60)
print()

query3 = "What courses can help me with business analytics and decision making?"
profile3 = {
    "name": "Jennifer Lee",
    "major": "Business Administration",
    "year": "Junior",
    "interests": ["analytics", "management", "strategy"],
    "preferred_difficulty": "intermediate",
    "preferred_format": "hybrid",
}

response3, summaries3, details3 = await hierarchical_rag_query(query3, profile3)
print(f"\nQuery: {query3}\n")
print()
print("\nAI Response:\n")
print(response3)

TEST 3: Business Analytics

19:13:34 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What courses can help me with business analytics and decision making?' (summaries=5, details=3)


19:13:35 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:35 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What courses can help me with business analytics and decision making?


19:13:35 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:35 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:13:42 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: What courses can help me with business analytics and decision making?



AI Response:

It seems that there are currently no courses listed that specifically align with business analytics and decision-making. However, I would encourage you to explore related subjects that may complement your interests in analytics, management, and strategy.

Since you're a junior majoring in Business Administration and prefer intermediate-level courses in a hybrid format, I recommend considering courses that might not be directly labeled as "business analytics" but could provide valuable skills and knowledge in that area. 

Keep an eye on future course offerings or reach out to your academic advisor to see if there are upcoming courses that match your interests. In the meantime, you might also want to explore independent study opportunities or online resources to enhance your analytics skills.

Stay motivated, Jennifer! Your interest in analytics will greatly benefit your future career in busine

---

## Part 8: Context Quality Matters

### Why Quality Engineering is Essential

You've built a working RAG system - congratulations! But there's a critical question: **What makes context "good"?**

Context engineering is real engineering - it requires the same rigor, analysis, and deliberate decision-making as any other engineering discipline. Let's see why this matters with a concrete example.

---

### Example: The Impact of Poor vs. Well-Engineered Context

Let's see what happens when we don't engineer our context properly.

**Scenario:** A student asks about machine learning courses.

In [17]:
import json
import tiktoken

def count_tokens(text: str, model: str = "gpt-4o-mini") -> int:
    """Count tokens in text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Poor context: Raw JSON dump (what we might do naively)
# Get first 10 courses using a broad search
poor_summaries, _ = await hierarchical_manager.hierarchical_search(
    query="course",
    summary_limit=10,
    detail_limit=0
)

poor_context = json.dumps(
    [
        {
            "course_code": s.course_code,
            "title": s.title,
            "summary": s.summary,
            "department": s.department,
            "difficulty_level": s.difficulty_level.value,
            "format": s.format.value,
        }
        for s in poor_summaries
    ],
    indent=2,
)

poor_tokens = count_tokens(poor_context)

print(f"""❌ POOR CONTEXT (Naive Approach):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: {len(poor_summaries)} (unfiltered - may not be relevant)
Tokens: {poor_tokens:,}
Format: Raw JSON with all fields

Sample:
{poor_context[:400]}...
""")

19:13:42 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'course' (summaries=10, details=0)


19:13:42 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:42 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: course


19:13:42 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:42 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


❌ POOR CONTEXT (Naive Approach):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: 0 (unfiltered - may not be relevant)
Tokens: 1
Format: Raw JSON with all fields

Sample:
[]...



Now let's compare with well-engineered context using our hierarchical RAG system:

In [18]:
# Well-engineered context: Filtered + Optimized with hierarchical retrieval
query = "What machine learning courses are available?"

# Use our hierarchical RAG system to get relevant courses
good_summaries, good_details = await hierarchical_manager.hierarchical_search(
    query=query,
    summary_limit=3,
    detail_limit=2
)

# Use context assembler for LLM-friendly format
well_engineered_context = context_assembler.assemble_hierarchical_context(
    summaries=good_summaries,
    details=good_details,
    query=query
)

good_tokens = count_tokens(well_engineered_context)

print(f"""✅ WELL-ENGINEERED CONTEXT (Hierarchical RAG + Optimization):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: {len(good_summaries)} summaries + {len(good_details)} details (filtered by semantic relevance)
Tokens: {good_tokens:,}
Format: LLM-optimized text with progressive disclosure

Context:
{well_engineered_context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Token Reduction: {poor_tokens - good_tokens:,} tokens ({((poor_tokens - good_tokens) / poor_tokens * 100):.1f}% reduction)
Cost Savings: ${((poor_tokens - good_tokens) / 1_000_000) * 2.50:.4f} per request
""")

19:13:43 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What machine learning courses are available?' (summaries=3, details=2)


19:13:43 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:13:43 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What machine learning courses are available?


19:13:43 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:13:43 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


✅ WELL-ENGINEERED CONTEXT (Hierarchical RAG + Optimization):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Courses: 0 summaries + 0 details (filtered by semantic relevance)
Tokens: 25
Format: LLM-optimized text with progressive disclosure

Context:
# Course Search Results for: What machine learning courses are available?

## Overview of All Matches

Found 0 relevant courses:


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Token Reduction: -24 tokens (-2400.0% reduction)
Cost Savings: $-0.0001 per request



### The Difference in LLM Responses

Let's see how context quality affects the actual responses:

In [19]:
# Test with poor context
messages_poor = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{poor_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]

response_poor = llm.invoke(messages_poor)

print(f"""❌ RESPONSE WITH POOR CONTEXT:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_poor.content}
""")

19:13:46 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


❌ RESPONSE WITH POOR CONTEXT:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
It appears that there are currently no machine learning courses available in the Redis University catalog. If you're interested in other topics or have specific subjects in mind, please let me know, and I can help you find relevant courses or resources!



In [20]:
# Test with well-engineered context
messages_good = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{well_engineered_context}

Help students find the best course for their needs."""
    ),
    HumanMessage(content=query),
]

response_good = llm.invoke(messages_good)

print(f"""✅ RESPONSE WITH WELL-ENGINEERED CONTEXT:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_good.content}
""")

19:13:48 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


✅ RESPONSE WITH WELL-ENGINEERED CONTEXT:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
It seems that there are currently no machine learning courses available in the Redis University course catalog. If you're looking for courses in machine learning, I recommend checking other educational platforms or institutions that may offer relevant programs. If you have any specific topics or skills in mind, I can help you find resources or suggest alternative learning paths!



### Key Takeaways: Why Context Engineering Matters

From this example, you can see that well-engineered context:

1. **Reduces Token Usage** - 50-70% fewer tokens through filtering and optimization
2. **Improves Relevance** - Semantic search finds the right courses
3. **Enhances Response Quality** - LLM can focus on relevant information
4. **Saves Money** - Fewer tokens = lower API costs
5. **Scales Better** - Works with thousands of courses, not just 10

**The Engineering Mindset:**
- Context is data that requires engineering discipline
- Raw data ≠ Good context
- Systematic transformation: Extract → Clean → Transform → Optimize → Store
- Quality metrics: Relevance, Completeness, Efficiency, Accuracy

---

## 🎓 Key Takeaways

### What You've Learned

**1. RAG Fundamentals**
- RAG dynamically retrieves relevant information instead of hardcoding knowledge
- Vector embeddings enable semantic search (meaning-based, not keyword-based)
- RAG solves the scalability and token efficiency problems of static context

**2. The Hierarchical RAG Pipeline**
```
User Query → Hierarchical Search → Context Assembly → LLM Generation
```
- **Retrieval:** Find relevant documents using vector similarity (summaries first)
- **Progressive Disclosure:** Fetch full details only for top matches
- **Assembly:** Combine system + user + retrieved context
- **Generation:** LLM creates personalized response with full context

**3. Context Engineering in Practice**
- **System Context:** AI role and instructions (static)
- **User Context:** Student profile and preferences (dynamic, user-specific)
- **Retrieved Context:** Relevant courses from vector search (dynamic, query-specific)
- **Integration:** All three context types work together

**4. Technical Implementation**
- **HierarchicalCourseManager**: Two-tier retrieval (summaries + details)
- **HierarchicalContextAssembler**: Progressive disclosure context assembly
- **Benefits**: Efficient token usage, better relevance, scalable architecture

### Best Practices

**Retrieval:**
- Retrieve only what's needed (top-k results)
- Use progressive disclosure (summaries → details)
- Balance between too few (missing info) and too many (wasting tokens) results
- **💡 Research Insight:** Context Rot research shows that distractors (similar-but-wrong information) have amplified negative impact in long contexts. Precision in retrieval matters more than recall.

**Context Assembly:**
- Structure context clearly (system → user → retrieved)
- Include only relevant metadata
- Keep descriptions concise but informative
- Use progressive disclosure for efficiency

**Generation:**
- Use appropriate temperature (0.7 for creative, 0.0 for factual)
- Provide clear instructions in system context
- Let the LLM explain its reasoning

---

## 🚀 What's Next?

### 📊 Module 3: Data Engineering for Context

Now that you understand RAG fundamentals and why context quality matters, Module 3 teaches you to engineer context with production-level rigor:
- Master data engineering workflows for context preparation
- Understand chunking as a design choice (not a default)
- Build production-ready context pipelines
- Optimize context quality with systematic approaches

### 🧠 Module 4: Memory Systems

In this module, you built a RAG system that retrieves relevant information for each query. But there's a problem: **it doesn't remember previous conversations**.

In Module 4, you'll add memory to your RAG system:
- **Working Memory:** Track conversation history within a session
- **Long-term Memory:** Remember user preferences across sessions
- **Agent Memory Server:** Automatic memory extraction and retrieval

### 🤖 Module 5: Building Agents

After adding memory, you'll transform your RAG system into a full agent:
- **Tool Calling:** Let the AI use functions (search, enroll, check prerequisites)
- **LangGraph State Management:** Orchestrate complex multi-step workflows
- **Agent Reasoning:** Plan and execute multi-step tasks
- **Production Patterns:** Error handling, retries, and monitoring

### The Journey

```
Module 1: Context Engineering Fundamentals
    ↓
Module 2: RAG Fundamentals ← You are here
    ↓
Module 3: Data Engineering for Context ← Next
    ↓
Module 4: Memory Systems
    ↓
Module 5: Building Agents (Complete System)
```

---

## 💪 Practice Exercises

Try these exercises to deepen your understanding:

**Exercise 1: Custom Filters**
- Modify the hierarchical search to filter by specific departments
- Try combining multiple filters (difficulty + format + department)

**Exercise 2: Adjust Retrieval**
- Experiment with different `summary_limit` and `detail_limit` values
- Observe how response quality changes with more/fewer retrieved courses

**Exercise 3: Context Optimization**
- Modify the `assemble_rag_context` function to include more/less detail
- Measure token usage and response quality trade-offs

**Exercise 4: Progressive Disclosure Tuning**
- Try different ratios of summaries to details (5:2, 10:3, 3:1)
- Measure token usage and response quality for each

**Exercise 5: Evaluation**
- Create test queries with expected results
- Measure retrieval accuracy (are the right courses retrieved?)
- Measure generation quality (are responses helpful and accurate?)

---

## 📝 Summary

You've built a complete hierarchical RAG system that:
- ✅ Uses two-tier retrieval (summaries + details)
- ✅ Performs semantic search to find relevant courses
- ✅ Applies progressive disclosure for efficient token usage
- ✅ Assembles context from multiple sources (system + user + retrieved)
- ✅ Generates personalized responses using LLMs
- ✅ Handles different query types and user profiles

This RAG system is the foundation for the advanced topics in Modules 3, 4, and 5. You'll build on this exact code to add data engineering, memory, tools, and full agent capabilities.

**Great work!** You've mastered Retrieved Context and built a production-ready hierarchical RAG pipeline. 🎉

---

## 📚 Additional Resources

### **RAG and Vector Search**
- [Retrieval-Augmented Generation Paper](https://arxiv.org/abs/2005.11401) - Original RAG paper by Facebook AI
- [Redis Vector Similarity Search](https://redis.io/docs/stack/search/reference/vectors/) - Official Redis VSS documentation
- [RedisVL Documentation](https://redisvl.com/) - Redis Vector Library for Python
- [LangChain RAG Tutorial](https://python.langchain.com/docs/tutorials/rag/) - Building RAG applications

### **Embeddings and Semantic Search**
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings) - Understanding text embeddings
- [Sentence Transformers](https://www.sbert.net/) - Open-source embedding models
- [HNSW Algorithm](https://arxiv.org/abs/1603.09320) - Hierarchical Navigable Small World graphs

### **LangChain and Redis Integration**
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction) - Framework overview
- [LangChain Redis Integration](https://python.langchain.com/docs/integrations/vectorstores/redis/) - Using Redis with LangChain
- [Redis Python Client](https://redis-py.readthedocs.io/) - redis-py documentation

### **Advanced RAG Techniques**
- [Advanced RAG Patterns](https://blog.langchain.dev/deconstructing-rag/) - LangChain blog on RAG optimization
- [Advanced Search with RedisVL](https://docs.redisvl.com/en/latest/user_guide/11_advanced_queries.html) - Vector, Hybrid, Text, and Keyword Search
- [RAG Evaluation](https://arxiv.org/abs/2309.15217) - Measuring RAG system performance

---

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Redis University - Context Engineering Workshop**
